In [1]:
from IPython.core.display import HTML  #For a more pleasing rendering...
HTML(open("styles/custom.css").read()) #When run in your local notebook.

## 2.0.1 Introduction to the topic and the algorithms 

<img src="./images/Ablauf_reinforcement_learning.PNG" width=400/>

<font>An **agent** exists in an **environment**, which evolves in discrete time steps. Agents can influence the environment by taking an **action** $a$ each time step, after which it receives a **reward** $r$ and an observed **state** $s$. For simplification, we only consider deterministic environments. That means that taking action $a$ in state $s$ always results in the same state $s’$.</font>

<font>Beyond the agent and the environment, one can identify three main subelements of
a reinforcement learning system: a policy, a reward signal and a value function.</font>

<font>A **policy** defines the learning agent’s way of behaving at a given time. Roughly
speaking, a policy is a mapping from perceived states of the environment to actions to
be taken when in those states.</font>

<font>A **reward signal** defines the goal in a reinforcement learning problem. On each time
step, the environment sends to the reinforcement learning agent a single number, a
reward. The agent’s sole objective is to maximize the total reward it receives over
the long run.</font>

<font>Whereas the reward signal indicates what is good in an immediate sense, a **value
function** specifies what is good in the long run. Roughly speaking, the value of
a state is the total amount of reward an agent can expect to accumulate over the
future, starting from that state. </font>

### Algorithm Types

<font>In our short introduction we will look at two different types of algorithms: [**Q-learning**](https://en.wikipedia.org/wiki/Q-learning) and [**Policy Gradient methods**](http://www.scholarpedia.org/article/Policy_gradient_methods), later on we present a state-of-the-art algorithm that utilizes both approaches. Good reads to all the topics can be found at the bottom of the [page](#Literature).</font>

### Q-learning

<font> Let’s define a function $Q(s, a)$ such that for given state $s$ and action $a$ it returns an estimate of a total reward we would achieve starting at this state, taking the action and then following some policy. There certainly exist policies that are optimal, meaning that they always select an action which is the best in the context. Let’s call the Q function for these optimal policies $Q^*$.
$$ $$
If we knew the true $Q^*$ function, the solution would be straightforward. We would just apply a greedy policy to it. That means that in each state s, we would just choose an action a that maximizes the function $Q^*$:  $argmax_a Q^*(s, a)$. Knowing this, our problem reduces to find a good estimate of the $Q^*$ function and apply the greedy policy to it.</font>

<font>Let’s write a formula for this function in a symbolic way. It is a sum of rewards we achieve after each action, but we will discount every partial sum with a $\gamma$:</font>

$$Q^*(s, a) = r_0 + \gamma r_1 + \gamma^2 r_2 + \gamma^3 r_3 + ...$$


  $\gamma$ <font>is called a discount factor. The $\gamma$ therefore controls how much the function $Q$ in state $s$ depends on the future and so it can be thought of as how much ahead the agent sees. Typically we set it to a value close, but lesser to one. The actions are chosen according to the greedy policy, maximizing the $Q^*$ function. When we look again at the formula, we see that we can write it in a recursive form: </font>

$$Q^*(s, a) = r_0 + \gamma (r_1 + \gamma r_2 + \gamma^2 r_3 + ...) = r_0 + \gamma \cdot max_a Q^*(s', a)$$

This is a so called [Bellman equation](https://en.wikipedia.org/wiki/Bellman_equation). 

<font>If we use the formula for our approximated function: $Q(s, a) = r + \gamma \cdot max_a Q(s', a)$. It was proven to converge to the desired $Q^*$, provided that there are finite number of states and each of the state-action pair is presented repeatedly. It means that we could use this assignment every time our agent experience a new transition and over time, it would converge to the Q* function.
$$ $$
However, in the problems we are trying to solve, the state usually consists of several real numbers and so our state space is infinite. We can’t obviously use any table to store infinite number of values. Instead, we will approximate the Q function with a neural network. This network will take a state as an input and produce an estimation of the Q function for each action.
$$ $$
But the original proof about the convergence does not hold anymore. Actually, the authors of the original research acknowledged that using a neural network to represent the Q function is known to be unstable. To face with this issue, they introduced several key ideas to stabilize the training, which are mainly responsible for the success we see. E.g. experience replay, exploration etc.  </font>

How you actually realise this ideas and how it's working will be shown in **[Chapter 2.1](2.1 Deep Q-Network.ipynb)**. 

### Policy Gradient

<font>Here: Agents actions are only determined by a **stochastic policy** $π(s)$. Stochastic policy means that it does not output a single action, but a distribution of probabilities over actions. We’ll also use a notation $π(a | s)$ which means the probability of taking action a in state s. </font>

<font>We define a value function $V(s)$ of policy $π$ as an expected discounted return, which can be viewed as a following recurrent definition:</font> 

$$V(s) = E_{\pi(s)}[ r + \gamma V(s') ] $$

Basically<font>, we weight-average the $r + \gamma V(s')$ for every possible action we can take in state s. $\gamma$ is our usual discount factor here. </font>

<font>Action-value function $Q(s, a)$ is on the other hand defined plainly as:</font>

$$Q(s, a) = r + \gamma V(s')$$

Simply<font> because the action is given and there is only one following $s’$.

Now, let’s define a new function $A(s, a)$ as:</font>

$$A(s, a) = Q(s, a) - V(s) $$

We<font> call $A(s, a)$ an advantage function and it expresses how good it is to take an action a in a state s compared to average. </font>

<font> The Q-Network tried to approximate the $Q(s, a)$ function. But now we will take a different approach. The policy $π$ is just a function of state $s$, so we can approximate directly that. Our neural network with weights $\theta$ will now take an state $s$ as an input and output an action probability distribution, $\pi_\theta$. From now on, by writing $π$ it is meant $\pi_\theta$, a policy parametrized by the network weights $\theta$.</font>

<font> In order to optimize a policy we need a metric that will tell us how good a policy is. Let’s define a function $J(\pi)$ as a **discounted reward** that a policy $π$ can gain, averaged over all possible starting states $s_0$.</font>

$$J(\pi) = E_{\rho^{s_0}}[V(s_0)]$$ 

$\rho$ <font> is the probability distribution of states to be in. $\rho^{s_0}$ is the distribution of starting states, $\rho^\pi$ is the distribution of states under the policy $\pi$. In other words, it gives us probabilities of being in a state when following policy $π$.
$$ $$
All together **$J$** just describes what reward we get in average for one episode with the policy $\pi$. 
$$ $$
The gradients of the function have the form: (see [[2]](#Literature) and [[4]](#Literature) for details) 

$$\nabla_\theta J(\pi) = E_{s\sim\rho^\pi, a \sim \pi(s)}\left[A(s,a) \cdot \nabla_\theta log(\pi(a|s) ) \right]$$</font>

<font>Fortunately, running an episode with a policy $π$ yields samples distributed exactly as we need. States encountered and actions taken are indeed an unbiased sample from the $\rho^\pi$ and $π(s)$ distributions.
$$ $$
We can simply let our agent run in the environment and record the $(s, a, r, s’)$ samples. When we gather enough of them, we use the formula above to find a good approximation of the gradient $\nabla_\theta\ J(\pi)$. We can then use any of the existing techniques based on gradient descend to improve our policy via backpropagation.</font>

For the technical aspects have a look at **[Chapter 2.2](2.2 Policy Gradient.ipynb)**.

#### Differences

<font>The Policy gradient method attempts to map an observation to the probabilitys of taking an action, while the Q-Learning attempts to learn the value of being in a specific state. So, while both can give us a good solution, they may differ from each other.</font>

### A3C

<font>Instead of only using samples to compute the advantage function $A(s,a)$, that we need to compute the gradients $\nabla_\theta J(\pi)$, we use a DQN to approximate the value function $V(s)$ just like we did before. </font>

<font>We can also use the same neural network for estimating $π(s)$ and $V(s)$. This has multiple benefits. Because we optimize both of these goals together, we learn much faster and effectively. </font>

<font>The samples we gather during a run of an agent are highly correlated. In DQN, we used a technique named Experience Replay to overcome this issue. We stored the samples in a memory and retrieved them in random order to form a batch.
$$ $$
But there’s another way to break this correlation; We can run several agents in parallel, each with its own copy of the environment, and use their samples as they arrive. Different agents will likely experience different states and transitions, thus avoiding the correlation. Another benefit is that this approach needs much less memory, because we don’t need to store the samples.
$$ $$
This is the approach the A3C algorithm takes. The full name is Asynchronous advantage actor-critic (A3C), more details about the infrastructure and two different simple implementations can be found in **[Chapter 2.3](2.3 Actor Critic Methods.ipynb)**.</font>

### Literature

<font> [1] [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/pdf/1312.5602v1.pdf): 
The original paper by DeepMind that started the process of learning RL algorithms on classic games. </font>
$$ $$
<font>
[2] [Reinforcement Learning: An Introduction - Richard S. Sutton and Andrew G. Barto](http://incompleteideas.net/sutton/book/bookdraft2017june19.pdf): Excellent book and reference text in general for Reinforcement Learning.  </font>
$$ $$
<font>[3] [Demystifying Deep Reinforcement Learning](http://neuro.cs.ut.ee/demystifying-deep-reinforcement-learning/): A good post about Q-learning for further explanation.   </font>
$$ $$
<font>[4] [Deep Reinforcement Learning: Pong from Pixels](http://karpathy.github.io/2016/05/31/rl/): Another beautiful blog post about Policy Gradients, which gets referenced very often. 
</font>
$$ $$
<font>[5] [Deep Reinforcement Learning: An Overview](https://arxiv.org/pdf/1701.07274.pdf): In this paper you will find a broad overview about RL, from choosing a model to reallife application nearly everything gets mentioned. </font>
